<a href="https://colab.research.google.com/github/aneuraz/intro-keras/blob/master/Hugging_Face_Transformers_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutoriel Hugging Face Transformers

> adapté de CS224N: Hugging Face Transformers Tutorial (Winter '22), Ben Newman ([version CS224N](https://colab.research.google.com/drive/1pxc-ehTtnVM72-NViET_D2ZqOlpOi2LH?usp=sharing))

Ce notebook vous donnera une introduction à la bibliothèque Python Hugging Face Transformers et à quelques modèles courants que vous pouvez utiliser pour en tirer parti. Cette librairie particulièrement utile pour utiliser ou fine-tuner des modèles transformers préentrainés dans vos projets.

Hugging Face fournit un accès aux modèles (à la fois le code qui les met en œuvre et leurs poids pré-entraînés), aux tokeniseurs spécifiques aux modèles, ainsi qu'aux pipelines pour les tâches NLP courantes et aux datasets et aux métriques. Il a des implémentations en PyTorch, Tensorflow et Flax (même si nous utiliserons ici les versions PyTorch !)

Nous allons passer en revue quelques cas d'utilisation :

- Aperçu des tokeniseurs et des modèles
- Finetuning - pour votre propre tâche. Nous utiliserons un exemple de classification de sentiments.

Nous pouvons distinguer différents types de projets de NLP:
1. Appliquer un modèle pré-entrainé à une nouvelle tâche ou un nouveau domaine et explorer des solutions pour résoudre cette question
2. Implémenter une nouvelle architecture de réseaux de neurones et démontrer ses performances sur des données.
3. Analyser le comportement d'un modèle: comment il représente les connaissances linguistiques ou quel type de phénomènes il peut gérer ou les erreurs qu'il commet. 

Parmi ces types de projets, `transformers` pourra être plus utile pour 1. et 3. (on peut aussi l'utiliser pour définir de nouvelles architectures mais c'est plus compliqué et nous n'aborderons pas cet aspect).

Voici quelques ressources complémentaires (en anglais) concernant la librairie `transformers`:

* [Hugging Face Docs](https://huggingface.co/docs/transformers/index)
  * documentation claire
  * Tutorials, pas-à-pas, et notebooks d'exemples
  * Liste des modèles disponibles
* [Cours Hugging Face](https://huggingface.co/course/)



In [ ]:
!pip install --quiet transformers datasets sentencepiece 


In [ ]:
#from collections import defaultdict, Counter
#import json

from matplotlib import pyplot as plt
import numpy as np
import torch

def print_encoding(model_inputs, indent=4):
    indent_str = " " * indent
    print("{")
    for k, v in model_inputs.items():
        print(indent_str + k + ":")
        print(indent_str + indent_str + str(v))
    print("}")

## Partie 0: Patterns courants pour l'utilisation de Hugging Face Transformers

Commençons par une utilisation commune de Transformers: l'analyse de sentiments. 

Premièrement, trouver un modèle sur [le hub](https://huggingface.co/models). Tout le monde peut uploader son modèle pour qu'il puisse être utilisé par tous. (Nous utiliserons le modèle d'analyse de sentiment issu de [ce repo](https://huggingface.co/citizenlab/twitter-xlm-roberta-base-sentiment-finetunned)).

Ensuite, deux objets différents doivent être initialisés: 
- un **tokenizer**: converti les textes en listes d'ids de vocabulaire (input ids) que le modèle nécessite
- un **modèle**: prend les ids de vocabulaire et produit une prédiction


![full_nlp_pipeline.png](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/full_nlp_pipeline.svg)
From [https://huggingface.co/course/chapter2/2?fw=pt](https://huggingface.co/course/chapter2/2?fw=pt)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "cmarkea/distilcamembert-base-sentiment"

# Initialise le tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Initialise le  model
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
inputs = "Je suis vraiment content de faire ce fabuleux tutoriel"
tokenized_inputs = tokenizer(inputs, return_tensors="pt")
outputs = model(**tokenized_inputs)
print(outputs)
labels = ['1 star','2 stars','3 stars','4 stars','5 stars']
prediction = torch.argmax(outputs.logits)


print("Input:")
print(inputs)
print()
print("Tokenized Inputs:")
print_encoding(tokenized_inputs)
print()
print("Model Outputs:")
print(outputs)
print()
print(f"La prédiction est {labels[prediction]}")

### 0.1 Tokenizers

Les modèles pré-entraînés sont mis en œuvre avec des **tokenizers** qui sont utilisés pour prétraiter leurs entrées. Les tokenizers prennent des chaînes brutes ou des listes de chaînes en entrée et produisent ce qui revient à des dictionnaires contenant les entrées du modèle.

Vous pouvez accéder aux tokenizers soit en utilisant la classe `Tokenize`
 spécifique au modèle que vous souhaitez utiliser (ici `DistilBert`), soit en utilisant la classe `AutoTokenizer`. Les tokenizers rapides sont écrits en Rust, tandis que leurs versions lentes sont écrites en Python.

In [ ]:
from transformers import CamembertTokenizer, CamembertTokenizerFast, AutoTokenizer

base_model = "cmarkea/distilcamembert-base"

tokenizer = CamembertTokenizer.from_pretrained(base_model)      # written in Python
print(tokenizer)
tokenizer = CamembertTokenizerFast.from_pretrained(base_model)  # written in Rust
print(tokenizer)
tokenizer = AutoTokenizer.from_pretrained(base_model) # convenient! Defaults to Fast
print(tokenizer)

In [ ]:
# Voici comment on appelle le tokinzer
input_str = "J'aime la glace à la vanille"
tokenized_inputs = tokenizer(input_str)


print("Vanilla Tokenization")
print_encoding(tokenized_inputs)
print()

# 2 méthodes pour récupérer les input_ids:
print(tokenized_inputs.input_ids)
print(tokenized_inputs["input_ids"])

In [ ]:
cls = [tokenizer.cls_token_id]
sep = [tokenizer.sep_token_id]

# La Tokenization se décompose en 3 étapes: 
input_tokens = tokenizer.tokenize(input_str)     
input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
input_ids_special_tokens = cls + input_ids + sep

decoded_str = tokenizer.decode(input_ids_special_tokens)

print("text de départ:                ", input_str)
print("tokenize:             ", input_tokens)
print("convert_tokens_to_ids:", input_ids)
print("add special tokens:   ", input_ids_special_tokens)
print("--------")
print("decode:               ", decoded_str)

# NOTE ces étapes ne créent par le masque d'attention (attention mask)

In [ ]:
# Pour les Fast Tokenizers,il y a une autre option également: 
inputs = tokenizer._tokenizer.encode(input_str)

print(input_str)
print("-"*5)
print(f"Number of tokens: {len(inputs)}")
print(f"Ids: {inputs.ids}")
print(f"Tokens: {inputs.tokens}")
print(f"Special tokens mask: {inputs.special_tokens_mask}")
print()
print("char_to_word retourne le wordpiece d'un caractère de l'input")
char_idx = 11
print(f"Par exemple, le {char_idx + 1}ème caractère de l'input est '{input_str[char_idx]}',"+\
      f" et il fait parti du wordpiece {inputs.char_to_token(char_idx)}, '{inputs.tokens[inputs.char_to_token(char_idx)]}'")

In [ ]:
# Autres propriétés intéressantes:
# Le tokenizer peut renvoyer directement des tensors pytorch
model_inputs = tokenizer("J'aime la glace à la vanille", return_tensors="pt")
print("PyTorch Tensors:")
print_encoding(model_inputs)

In [ ]:
# On peut passer plusieurs strings à la fois au tokenizer et effectuer un padding en même temps:
model_inputs = tokenizer(["J'aime la glace à la vanille",
                         "Pour le Parlement européen, l'intelligence artificielle " +\
                         "représente tout outil utilisé par une machine afin de" +\
                         "« reproduire des comportements liés aux humains. "
                         ],
                         return_tensors="pt",
                         padding=True,
                         truncation=True)
print(f"Pad token: {tokenizer.pad_token} | Pad token id: {tokenizer.pad_token_id}")
print("Padding:")
print_encoding(model_inputs)

In [ ]:
# Il est également possible de décoder un batch entier:
print("Batch Decode:")
print(tokenizer.batch_decode(model_inputs.input_ids))
print()
print("Batch Decode: (no special characters)")
print(tokenizer.batch_decode(model_inputs.input_ids, skip_special_tokens=True))

Pour plus d'informations, vous pouvez aller voir sur: 
[la documentation Hugging Face Transformers](https://huggingface.co/docs/transformers/main_classes/tokenizer) et la [librairie Hugging Face Tokenizers](https://huggingface.co/docs/tokenizers/python/latest/quicktour.html) (pour les Fast Tokenizers). La librairie Tokenizers vous permet également d'entrainer vos propres tokenizers!

### 0.2 Modèles

Initialiser un modèle est très similaire à l'initialisation d'un tokenizer. Vous pouvez utiliser une classe spécifique au modèle ou passer par la classe AutoModel. Il peut être utile de privilégier AutoModel car si l'on veut comparer des modèles de classes différentes, cela évite d'alourdir le code et il suffit de changer le nom du modèle dans le code. 

La plupart des modèles transformers préentrainés ont une architecture similaire, il faut ajouter une tête (head) à entrainer en fonction de la tâche à effectuer: classification de séquence, question answering, classification de tokens, ... 
Par exemple, nous voulons faire de l'analyse de sentiment. Il s'agit d'une tâche de classification de séquence, nous avons donc besoin de la classe `XLMRobertaForSequenceClassification`. Si nous voulions continuer d'entrainer XLMRoberta sur sa tâche de masked-language modelling, nous pourrions utiliser `XLMRobertaForMaskedLM`, et si nous voulions uniquement récupérer la représentation issue du modèle nous pourrions utiliser `XLMRobertaModel`. 


Voici une illustration de l'architecture des modèles dans HF Transformers: 
![model_illustration.png](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/transformer_and_head.svg)
source: [https://huggingface.co/course/chapter2/2?fw=pt](https://huggingface.co/course/chapter2/2?fw=pt).


Voici quelques examples de têtes spécialisées.
```
*
*ForMaskedLM
*ForSequenceClassification
*ForTokenClassification
*ForQuestionAnswering
*ForMultipleChoice
...
```
où `*` peut être `AutoModel` ou un modèle spécifique (e.g. `DistilBert`)

Il existe 3 types de modèles: 
* Encodeurs (e.g. BERT)
* Decodeurs (e.g. GPT2)
* Modèles Encodeur-Decodeur (e.g. BART or T5)

En fonction du type de modèle, les tâches supportées et donc les classes associées peuvent changer.

La liste complète des choix possibles est disponible dans la [documentation] (https://huggingface.co/docs/transformers/model_doc/auto). Attention, tous les modèles ne ne sont pas compatibles avec toutes les architectures. Par exemple, DistilBERT n'est pas compatible avec les modèles Seq2seq car c'est un modèle constitué uniquement d'un encodeur. 


In [ ]:
from transformers import AutoModelForSequenceClassification


model = AutoModelForSequenceClassification.from_pretrained(base_model, num_labels=5)


Un Warning apparait car les paramètres de la tête de classification de séquence n'ont pas encore été entrainés.

Passer les inputs au modèle est très simple. Les modèles prennent les inputs en 
keyword argument.

In [ ]:
model_inputs = tokenizer(input_str, return_tensors="pt")

# Option 1
model_outputs = model(input_ids=model_inputs.input_ids, 
                      attention_mask=model_inputs.attention_mask)

# Option 2 - Les clés du dictionnaire sont les mêmes que les arguments attendus par le modèle

# f({k1: v1, k2: v2}) = f(k1=v1, k2=v2)

model_outputs = model(**model_inputs)

print(model_inputs)
print()
print(model_outputs)
print()
print(f"Distribution over labels: {torch.softmax(model_outputs.logits, dim=1)}")

These models are just Pytorch Modules! You can can calculate the loss with your `loss_func` and call `loss.backward`. You can use any of the optimizers or learning rate schedulers that you used

In [ ]:
# You can calculate the loss like normal
label = torch.tensor([1])
loss = torch.nn.functional.cross_entropy(model_outputs.logits, label)
print(loss)
loss.backward()

# You can get the parameters
list(model.named_parameters())[0]

Hugging Face provides an additional easy way to calculate the loss as well:

In [ ]:
# To calculate the loss, we need to pass in a label:
model_inputs = tokenizer(input_str, return_tensors="pt")

model_inputs['labels'] = torch.tensor([1])

model_outputs = model(**model_inputs)


print(model_outputs)
print()
print(f"Model predictions: {labels[model_outputs.logits.argmax()]}")

One final note - you can get the hidden states and attention weights from the models really easily. This is particularly helpful if you're working on an analysis project. (For example, see [What does BERT look at?](https://arxiv.org/abs/1906.04341)).

In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained(base_model, output_attentions=True, output_hidden_states=True)
model.eval()

model_inputs = tokenizer(input_str, return_tensors="pt")
with torch.no_grad():
    model_output = model(**model_inputs)


print("Hidden state size (per layer):  ", model_output.hidden_states[0].shape)
print("Attention head size (per layer):", model_output.attentions[0].shape)     # (layer, batch, query_word_idx, key_word_idxs)
                                                                               # y-axis is query, x-axis is key
print(model_output)    

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(model_inputs.input_ids[0])
print(tokens)
n_tokens = len(tokens)

n_layers = len(model_output.attentions)
n_heads = len(model_output.attentions[0][0])
fig, axes = plt.subplots(n_layers, n_heads)
fig.set_size_inches(18.5*2, 10.5*2)
for layer in range(n_layers):
    for i in range(n_heads):
        axes[layer, i].imshow(model_output.attentions[layer][0, i])
        axes[layer][i].set_xticks(list(range(n_tokens)))
        axes[layer][i].set_xticklabels(labels=tokens, rotation="vertical")
        axes[layer][i].set_yticks(list(range(n_tokens)))
        axes[layer][i].set_yticklabels(labels=tokens)

        if layer == n_layers:
            axes[layer, i].set(xlabel=f"head={i}")
        if i == 0:
            axes[layer, i].set(ylabel=f"layer={layer}")
            
plt.subplots_adjust(wspace=0.1)
plt.show()

## Part 1: Finetuning

Vous allez surement devoir finetuner un modèle préentrainé pour vos projets.




### 2.1 Charger un dataset

En plus des modèles, on peut trouver des datasets sur [le hub](https://huggingface.co/datasets) Hugging Face.


In [ ]:
from datasets import load_dataset, DatasetDict

amazon_dataset = load_dataset("amazon_reviews_multi", "fr")

# Prenons uniquement les 128 premiers caractères pour accélerer l'entrainement.
def truncate(example):
    return {
        'review_body': " ".join(example['review_body'].split()[:128]),
        'label': example['stars']-1
    }

# Prend 1280 examples aléatoires pour le train et 320 pour la validation
small_amazon_dataset = DatasetDict(
    train=amazon_dataset['train'].shuffle(seed=1111).select(range(1280)).map(truncate),
    val=amazon_dataset['train'].shuffle(seed=1111).select(range(1280, 1600)).map(truncate),
)

In [ ]:
small_amazon_dataset['train'][:10]

In [ ]:
# Prepare le dataset - tokenize le dataset en batches de 32 exemples.
small_tokenized_dataset = small_amazon_dataset.map(
    lambda example: tokenizer(example['review_body'], padding=True, truncation=True),
    batched=True,
    batch_size=32
)

small_tokenized_dataset = small_tokenized_dataset.remove_columns(["review_body", 'review_id', 'product_id', 'reviewer_id', 'stars','review_title', 'language', 'product_category'])
small_tokenized_dataset = small_tokenized_dataset.rename_column("label", "labels")
small_tokenized_dataset.set_format("torch")

In [ ]:
small_tokenized_dataset['train'][0:2]

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_tokenized_dataset['train'], batch_size=32)
eval_dataloader = DataLoader(small_tokenized_dataset['val'], batch_size=32)

### 2.2 Entrainement

Pour entrainer vos modèles, vous pouvez utiliser une approche Pytorch classique puisque les modèles Hugging Face sont des modules Pytorch standards. Hugging Face propose également un Trainer qui simplifie l'entrainement.



`TrainingArguments` spécifie differents paramètres d'entrainerment comme le fréquence des évaluations, où sauver les checkpoints, ... 
De nombreux aspects peuvent être customizés, vous pouvez trouver plus de renseignements [ici](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments). 
Parmi les paramètres vous trouverez:
* learning rate, weight decay, gradient clipping, 
* checkpointing, logging, and evaluation frequency
* destination du log (par défaut tensorboard, mais vous pouvez aussi utiliser WandB ou MLFlow )

Le `Trainer` effectue l'entrainement. Vous pouvez lui passer les `TrainingArguments`, un modèle, les datasets, le tokenizer, l'optimizer, et même des model checkpoints pour reprendre un entrainement. La fonction `compute_metrics` est appelée à la fin pour calculer la métrique d'évaluation.

In [ ]:
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(base_model, num_labels=5)

arguments = TrainingArguments(
    output_dir="sample_hf_trainer",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=4,
    evaluation_strategy="epoch", # validation à la fin de chaque epoch
    save_strategy="epoch",
    learning_rate=2e-5,
    load_best_model_at_end=True,
    seed=224
)


def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return {"accuracy": np.mean(predictions == labels)}


trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['val'], # changer à test pour l'évaluation finale
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

#### Callbacks: Logging and Early Stopping


Hugging Face Transformers also allows you to write `Callbacks` if you want certain things to happen at different points during training (e.g. after evaluation or after an epoch has finished). For example, there is a callback for early stopping, and I usually write one for logging as well.

For more information on callbacks see [here](https://huggingface.co/docs/transformers/main_classes/callback#transformers.TrainerCallback).

In [ ]:
from transformers import TrainerCallback, EarlyStoppingCallback

class LoggingCallback(TrainerCallback):
    def __init__(self, log_path):
        self.log_path = log_path
        
    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero:
            with open(self.log_path, "a") as f:
                f.write(json.dumps(logs) + "\n")


trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=1, early_stopping_threshold=0.0))
#trainer.add_callback(LoggingCallback("sample_hf_trainer/log.jsonl"))

In [ ]:
# train the model
trainer.train()

In [ ]:
# evaluer le modèle est très simple

 results = trainer.evaluate()                           # retourne les métriques d'évaluation
results = trainer.predict(small_tokenized_dataset['val']) # avec les prédictions aussi

In [ ]:
results

In [ ]:
finetuned_model = AutoModelForSequenceClassification.from_pretrained("sample_hf_trainer/checkpoint-160")
finetuned_tokenizer = AutoTokenizer.from_pretrained("sample_hf_trainer/checkpoint-160")

model_inputs = finetuned_tokenizer("j'ai aimé ce film", return_tensors="pt")
prediction = torch.argmax(finetuned_model(**model_inputs).logits)
print(labels[prediction])

In [ ]:
from transformers import TextClassificationPipeline

pipe = TextClassificationPipeline(model=finetuned_model, tokenizer=finetuned_tokenizer)
pipe(["j'ai aimé ce film", "c'était pas terrible quand même"])

Quelques astuces pour le finetuning

**Bons hyperparameters par défaut.** Les hyperparamètres dépendent de la tâche et du dataset. Vous devrez faire une recherche des meilleurs hyperparamètres pour trouver les bons. Voici des bonnes baselines pour commencer: 
* Epochs: {2, 3, 4} (plus le dataset est grand et moins on fait d'epochss)
* Batch size (le plus grand possible est le mieux )
* Optimizer: AdamW
* AdamW learning rate: {2e-5, 5e-5}
* Learning rate scheduler: linear warm up for first {0, 100, 500} steps of training
* weight_decay (l2 regularization): {0, 0.01, 0.1}

Vous devriez monitorer la validation loss pour décider quand vous avez trouvé les bons hyperparamètres.


Le Trainer peut être largement personalisé. voir [ce lien](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer)


## Appendix 1: Datasets personalisés

Il existe plusieurs façons pour définier des datasets. Nous allons voir un exemple qui utilise les Pytorch Dataloaders.


In [ ]:
# Option 1: Load into Hugging Face Datasets

import pandas as pd
from datasets import Dataset

df = pd.read_json("https://raw.githubusercontent.com/aneuraz/casCliniques/main/casCliniques/trainset.json")
custom_dataset = Dataset.from_pandas(df)

In [ ]:
custom_dataset[0]

In [ ]:
import csv
import json, urllib.request
from torch.utils.data import Dataset, DataLoader

class CCDataset(Dataset):
    """Tokenize data when we call __getitem__"""
    def __init__(self, path, tokenizer):
        with urllib.request.urlopen(path) as f:
            reader = json.load(f)
            self.data = [{"source": row['token'], "target": row["label"]} for row in reader]
        self.tokenizer = tokenizer
            
    def __getitem__(self, i):
        inputs = self.tokenizer(self.data[i]['source'])
        labels = self.tokenizer(self.data[i]['target'])
        inputs['labels'] = labels.input_ids
        return inputs


In [ ]:
bart_tokenizer = AutoTokenizer.from_pretrained('facebook/bart-base')

In [ ]:
dataset = CCDataset("https://raw.githubusercontent.com/aneuraz/casCliniques/main/casCliniques/trainset.json", bart_tokenizer)

In [ ]:
bart_tokenizer.prepare_seq2seq_batch(src_texts=["This is the first test.", "This is the second test."], tgt_texts=["Target 1", "Target 2"], return_tensors="pt")

In [ ]:
dataset[0]